# Berechnung der Durchschnittlichen Spiellänge mittels Markov-Ketten
### Import der benötigten Bibliotheken

In [1]:
import sympy as sy


### Definition der Wahrscheinlichkeiten der einzelnen Würfelwerte


In [45]:
VierfachMünzwurf = {
    "dice": sy.nsimplify(1/16, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(3/8, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(1/16, tolerance=0.001, rational=True)
}
D4 = {
    "dice": sy.nsimplify(0, tolerance=0.001, rational=True),
    "diceI": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIII": sy.nsimplify(1/4, tolerance=0.001, rational=True),
    "diceIV": sy.nsimplify(1/4, tolerance=0.001, rational=True),
}

### Definition der Übergangsmatrizen

In [46]:
def RundenMatrix(dice,diceI,diceII,diceIII,diceIV):
    return sy.Matrix(
        [[dice,  diceI,  diceII,  diceIII,  diceIV * dice,  diceIV * diceI,  diceIV * diceII,  diceIV * diceIII,  diceIV * diceIV * dice,  diceIV * diceIV * diceI,  diceIV * diceIV * diceII,  diceIV * diceIV * diceIII,  diceIV * diceIV * diceIV, 0, 0, 0],
        [0,  dice,  diceI,  diceII,  diceIII * dice,  diceIV + diceIII * diceI,  diceIII * diceII,  diceIII * diceIII,  diceIII * diceIV *
            dice,  diceIII * diceIV * diceI,  diceIII * diceIV * diceII,  diceIII * diceIV * diceIII,  diceIII * diceIV * diceIV, 0, 0, 0],
            [0, 0,  dice,  diceI,  diceII * dice,  diceIII + diceII * diceI,  diceIV + diceII * diceII,  diceII * diceIII,  diceII * diceIV *
                dice,  diceII * diceIV * diceI,  diceII * diceIV * diceII,  diceII * diceIV * diceIII,  diceII * diceIV * diceIV, 0, 0, 0],
            [0, 0, 0,  dice,  diceI * dice,  diceII + diceI * diceI,  diceIII + diceI * diceII,  diceIV + diceI * diceIII,  diceI *
                diceIV * dice,  diceI * diceIV * diceI,  diceI * diceIV * diceII,  diceI * diceIV * diceIII,  diceI * diceIV * diceIV, 0, 0, 0],
            [0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII,  diceIV * dice,  diceIV *
                diceI,  diceIV * diceII,  diceIV * diceIII,  diceIV * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII * dice,  diceIV + diceIII *
                diceI,  diceIII * diceII,  diceIII * diceIII,  diceIII * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII * dice,  diceIII + diceII * diceI,
                diceIV + diceII * diceII,  diceII * diceIII,  diceII * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0,  dice,  diceI * dice,  diceII + diceI * diceI, diceIII +
                diceI * diceII,  diceIV + diceI * diceIII,  diceI * diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,
                diceII,  diceIII,  diceIV, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI, diceII,  diceIII,  diceIV, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII,
                diceIV*(dice + diceII + diceIII + diceIV),  diceIV * diceI],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice,  diceI,  diceII,  diceIII *
                (dice + diceII + diceIII + diceIV),  diceIV + (diceIII * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice + diceIV,  diceI,  diceII *
                (dice + diceII + diceIII + diceIV),  diceIII + (diceII * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  dice + diceIII + diceIV,
                diceI*(dice + diceII + diceIII + diceIV),  diceII + (diceI * diceI)],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                dice + diceII + diceIII + diceIV,  diceI],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


def SchritteMatrix(dice, diceI, diceII, diceIII, diceIV):
    return  sy.Matrix([
    [dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, dice, diceI,  diceII,  diceIII,  diceIV, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII,  diceIII, diceIV, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII,  diceIV, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice,  diceI,  diceII, diceIII, diceIV],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice+diceIV,  diceI,  diceII,  diceIII],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice+diceIII + diceIV,  diceI,  diceII],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice+diceII + diceIII + diceIV,  diceI],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, dice + diceI + diceII + diceIII + diceIV],
])


### Festlegen welcher Würfel verwendet wird

In [47]:
M_S = SchritteMatrix(**VierfachMünzwurf)
M_R = RundenMatrix(**VierfachMünzwurf)
# M_S = SchritteMatrix(**D4)
# M_R = RundenMatrix(**D4)


### Überprüfung ob die Zeilensumme jeweils 1 ist

In [48]:
print("Zeilensumme Schritte Matrix:", [sum(M_S.row(i)) for i in range(16)])
print("Zeilensumme Runden Matrix:",[sum(M_R.row(i)) for i in range(16)])

Zeilensumme Schritte Matrix: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Zeilensumme Runden Matrix: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Löschen der letzten Spalte
Achtung findet inplace statt!

In [49]:
M_S.col_del(-1)
M_R.col_del(-1)


### Löschen der letzten Zeile

Achtung findet inplace statt!

In [50]:
M_S.row_del(-1)
M_R.row_del(-1)


### Definition der 15x15 - Einheitsmatrix

In [51]:
I = sy.eye(15)

### Berechnung der Ergebnismatrizen

In [52]:
E_S = (I-M_S)**-1
E_R = (I-M_R)**-1

### Berechnng der jeweils ersten Zeilensumme

In [53]:
v=sy.Matrix([[1]+[0]*14])
ZS_S= sum(v*E_S)
ZS_R= sum(v*E_R)


## Ausgabe der Ergebnisse

In [54]:
print("Schritte gerundet:", float(ZS_S))
print("Schritte exakt für Latex:", sy.latex(ZS_S))
print("Schritte exakt:")

ZS_S




Schritte gerundet: 9.867846767445288
Schritte exakt für Latex: \frac{1280316980941868}{129746337890625}
Schritte exakt:


1280316980941868/129746337890625

In [55]:
print("Runden gerundet:", float(ZS_R))
print("Runden exakt für Latex:", sy.latex(ZS_R))
print("Runden exakt:")
ZS_R

Runden gerundet: 8.391927856222578
Runden exakt für Latex: \frac{1088821907187203}{129746337890625}
Runden exakt:


1088821907187203/129746337890625

In [56]:
print("Schritte/Runden gerundet:", float(ZS_S/ZS_R))
print("Schritte/Runden exakt für Latex:", sy.latex(ZS_S/ZS_R))
print("Schritte/Runden exakt:")
ZS_S/ZS_R


Schritte/Runden gerundet: 1.175873641493274
Schritte/Runden exakt für Latex: \frac{1280316980941868}{1088821907187203}
Schritte/Runden exakt:


1280316980941868/1088821907187203